In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/content/Data/train.csv") 

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
# Dropping the Null Values

df =df.dropna()

In [5]:
# Taking out the dependent featire label
X=df.drop('label',axis=1)


In [6]:
# Storing the label variable in y
y = df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
import tensorflow as tf

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [11]:
# Defining vocabulary
voc_size = 5000

### OneHot Represantation

In [12]:
msg = X.copy()

In [13]:
# Ressting index as we have dropped the NUll vlaues 
msg.reset_index(inplace=True)

In [14]:
import nltk
import re
from nltk.corpus import stopwords

In [15]:

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
# Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(0,len(msg)):
  #print(i)
  review = re.sub('[^a-zA-Z]',' ',msg['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)



# Substituting all the characters by " " except [^a-zA-Z]

In [17]:
corpus[1:5]

['flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [18]:
# onehot Represantation
onehot  = [one_hot(words,voc_size)for words in corpus]
max(onehot)

[4999, 115]

### Embedding Representation

In [19]:
length = 20
docs = pad_sequences(onehot,padding='pre',maxlen=length)
docs

array([[   0,    0,    0, ..., 4850, 4505,  203],
       [   0,    0,    0, ..., 4140, 4855, 4268],
       [   0,    0,    0, ...,  893,   42,  868],
       ...,
       [   0,    0,    0, ...,  640, 3512, 1428],
       [   0,    0,    0, ...,  237,  903, 1344],
       [   0,    0,    0, ...,  164,  357, 3096]], dtype=int32)

In [20]:

## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=length))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
# Converting the data to array
import numpy as np
X_final=np.array(docs)
y_final=np.array(y)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [23]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 50ms/step - loss: 0.2929 - accuracy: 0.8611 - val_loss: 0.1956 - val_accuracy: 0.9168
Epoch 2/10
192/192 [==============================] - 9s 48ms/step - loss: 0.1402 - accuracy: 0.9469 - val_loss: 0.2121 - val_accuracy: 0.9155
Epoch 3/10
192/192 [==============================] - 9s 48ms/step - loss: 0.1011 - accuracy: 0.9624 - val_loss: 0.2230 - val_accuracy: 0.9142
Epoch 4/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0738 - accuracy: 0.9736 - val_loss: 0.2467 - val_accuracy: 0.9118
Epoch 5/10
192/192 [==============================] - 9s 49ms/step - loss: 0.0495 - accuracy: 0.9826 - val_loss: 0.3566 - val_accuracy: 0.9109
Epoch 6/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0366 - accuracy: 0.9885 - val_loss: 0.3219 - val_accuracy: 0.9118
Epoch 7/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0271 - accuracy: 0.9916 - val_loss: 0.3459 - val_accuracy: 0.904

In [26]:
# prediction
y_pred=model.predict_classes(X_test)

In [27]:
# Confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_pred)

array([[3103,  316],
       [ 241, 2375]])

In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.907705053852527

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



In [31]:
y_pred.shape

(6035, 1)